# C_5choose_learning_machine_models with SMOTE


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#load the file 
data = pd.read_csv('../data/cleaned_data/3experiment_re_cleaned_data.csv')
data.head()


,monthly_charges,total_charges,senior_citizen_No,senior_citizen_Yes,partner_No,partner_Yes,dependents_No,dependents_Yes,multiple_lines_No,multiple_lines_No phone service,...,contract_Month-to-month,contract_One year,contract_Two year,paperless_billing_No,paperless_billing_Yes,payment_method_Bank transfer (automatic),payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,churn
0,-1.159996,-0.994194,1,0,0,1,1,0,0,1,...,1,0,0,0,1,0,0,1,0,No
1,-0.241345,-0.173740,1,0,1,0,1,0,1,0,...,0,1,0,1,0,0,0,0,1,No
2,-0.344532,-0.959649,1,0,1,0,1,0,1,0,...,1,0,0,0,1,0,0,0,1,Yes
3,-0.732875,-0.195248,1,0,1,0,1,0,0,1,...,0,1,0,1,0,1,0,0,0,No
4,0.211884,-0.940457,1,0,1,0,1,0,1,0,...,1,0,0,0,1,0,0,1,0,Yes


In [3]:
data.columns

Index(['monthly_charges', 'total_charges', 'senior_citizen_No',
       'senior_citizen_Yes', 'partner_No', 'partner_Yes', 'dependents_No',
       'dependents_Yes', 'multiple_lines_No',
       'multiple_lines_No phone service', 'multiple_lines_Yes',
       'internet_service_DSL', 'internet_service_Fiber optic',
       'internet_service_No', 'online_security_No',
       'online_security_No internet service', 'online_security_Yes',
       'online_backup_No', 'online_backup_No internet service',
       'online_backup_Yes', 'device_protection_No',
       'device_protection_No internet service', 'device_protection_Yes',
       'tech_support_No', 'tech_support_No internet service',
       'tech_support_Yes', 'streaming_tv_No',
       'streaming_tv_No internet service', 'streaming_tv_Yes',
       'streaming_movies_No', 'streaming_movies_No internet service',
       'streaming_movies_Yes', 'contract_Month-to-month', 'contract_One year',
       'contract_Two year', 'paperless_billing_No', 'paper

In [4]:
data = data[['monthly_charges', 'total_charges', 'payment_method_Bank transfer (automatic)', 'payment_method_Credit card (automatic)',
             'payment_method_Electronic check','payment_method_Mailed check', 'churn', 'contract_Two year','contract_One year',
             'contract_Month-to-month'
             
            ]]


### x/y split

In [5]:
#separate the features from the labels
y = data['churn']
X = data.drop(['churn'], axis=1)

In [6]:
# change the type of 'y'
y.dtypes

dtype('O')

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
display(y_train.value_counts())
display(y_test.value_counts())

churn
No     4130
Yes    1495
Name: count, dtype: int64

churn
No     1033
Yes     374
Name: count, dtype: int64

#### It is imbalanced,so it needs up/down sampling or SMOTE

In [9]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
# Fit and apply SMOTE to generate synthetic samples for the minority class
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [10]:
X_resampled.shape

(8260, 9)

In [11]:
class_counts = y_resampled.value_counts()
print(class_counts)

churn
Yes    4130
No     4130
Name: count, dtype: int64


In [12]:
# save 
#X_resampled.to_csv('../data/x_y_data/re_smote/experiment_X_resampled.csv', index=False)
#y_resampled.to_csv('../data/x_y_data/re_smote/experiment_y_resampled.csv', index=False)
#X_test.to_csv('../data/x_y_data/re_smote/experiment_X_test.csv', index=False)
#y_test.to_csv('../data/x_y_data/re_smote/experiment_y_test.csv', index=False)

### Logistic

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, classification_report

In [14]:


lg1= LogisticRegression(penalty = 'l2')

lg_tuned = lg1.fit(X_train, y_train)
lg1.fit(X_train, y_train)

y_pred = lg1.predict(X_test)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
display(cm)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score


y_pred = lg1.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='Yes')
recall = recall_score(y_test, y_pred, pos_label='Yes')
f1 = f1_score(y_test, y_pred, pos_label='Yes')
kappa = cohen_kappa_score(y_test, y_pred)

# Create a DataFrame
metrics_df = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1-Score", "Kappa"],
    "Score": [accuracy, precision, recall, f1, kappa]
})

display(metrics_df)

Confusion Matrix:


array([[919, 114],
       [199, 175]])

,Metric,Score
0,Accuracy,0.777541
1,Precision,0.605536
2,Recall,0.467914
3,F1-Score,0.527903
4,Kappa,0.385503
